<a href="https://colab.research.google.com/github/A1ienSword/Pattern-recognition-labs/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_9_%D0%9A%D0%BE%D1%81%D1%82%D0%B8%D1%86%D1%8B%D0%BD_%D0%92%D0%92_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!unzip dataset.zip

Archive:  dataset.zip
replace test_comp/F/0026.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test_comp/F/0026.png    
  inflating: test_comp/F/0022.png    
  inflating: test_comp/F/0020.png    
  inflating: test_comp/F/0018.png    
  inflating: test_comp/F/0003.png    
  inflating: test_comp/F/0017.png    
  inflating: test_comp/F/0019.png    
  inflating: test_comp/F/0023.png    
  inflating: test_comp/F/0014.png    
  inflating: test_comp/F/0025.png    
  inflating: test_comp/F/0000.png    
  inflating: test_comp/F/0001.png    
  inflating: test_comp/F/0011.png    
  inflating: test_comp/F/0008.png    
  inflating: test_comp/F/0010.png    
  inflating: test_comp/F/0021.png    
  inflating: test_comp/F/0002.png    
  inflating: test_comp/F/0005.png    
  inflating: test_comp/F/0027.png    
  inflating: test_comp/F/0028.png    
  inflating: test_comp/F/0016.png    
  inflating: test_comp/F/0013.png    
  inflating: test_comp/F/0029.png    
  inflating: test_comp/F/0024.png 

In [18]:
import os
import json
from PIL import Image
import numpy as np
from tqdm import tqdm

In [32]:
# Константы направлений
DIRECTIONS = {
    (-1, 0): '1',   # вверх
    (-1, 1): '2',   # вверх-вправо
    (0, 1): '3',    # вправо
    (1, 1): '4',    # вниз-вправо
    (1, 0): '5',    # вниз
    (1, -1): '6',   # вниз-влево
    (0, -1): '7',   # влево
    (-1, -1): '8'   # вверх-влево
}

def find_start_point(img_array):
    """Найти стартовую точку (самая левая нижняя)"""
    for col in range(img_array.shape[1]):
        for row in reversed(range(img_array.shape[0])):
            if img_array[row, col] == 1:
                return (row, col)
    return None

def get_neighbors(point, img_array):
    """Получить соседние точки по часовой стрелке"""
    row, col = point
    neighbors = []
    for (dr, dc), code in DIRECTIONS.items():
        nr, nc = row + dr, col + dc
        if 0 <= nr < img_array.shape[0] and 0 <= nc < img_array.shape[1]:
            if img_array[nr, nc] == 1:
                neighbors.append((nr, nc))
    return neighbors

def trace_image(img_array):
    """Построить сигнатуру изображения"""
    start = find_start_point(img_array)
    if not start:
        return ""

    visited = set()
    stack = [(start, None)]
    path = []
    branches = []

    while stack:
        current, parent = stack.pop()
        if current in visited:
            continue
        visited.add(current)

        if parent:
            dr = current[0] - parent[0]
            dc = current[1] - parent[1]
            path.append(DIRECTIONS[(dr, dc)])

        neighbors = get_neighbors(current, img_array)
        unvisited = [n for n in neighbors if n not in visited]

        if len(unvisited) > 1:
            path.append('(')
            branches.extend(unvisited[1:])

        for neighbor in reversed(unvisited[:1]):
            stack.append((neighbor, current))

        if not unvisited and branches:
            path.append(')(')
            next_point = branches.pop()
            stack.append((next_point, None))

    return ''.join(path)

def normalize_signature(signature):
    """Упрощение сигнатуры"""
    # Удаление дубликатов
    for i in range(1, 9):
        signature = signature.replace(str(i)*2, str(i))
    # Примеры замен комбинаций
    signature = signature.replace("121", "2").replace("212", "2")
    return signature

def train_model(data_dir, model_path):
    """Обучение модели с прогресс-баром"""
    model = {}

    # Считаем общее количество изображений
    total_images = 0
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            total_images += len(os.listdir(class_dir))

    # Создаем прогресс-бар
    with tqdm(total=total_images, desc="Обучение модели", unit="img") as pbar:
        for class_name in os.listdir(data_dir):
            class_dir = os.path.join(data_dir, class_name)
            if not os.path.isdir(class_dir):
                continue

            model[class_name] = []
            for img_file in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_file)
                img = Image.open(img_path).convert('L')
                img_array = (np.array(img) > 128).astype(int)
                signature = trace_image(img_array)
                model[class_name].append(normalize_signature(signature))

                # Обновляем прогресс-бар
                pbar.update(1)

    with open(model_path, 'w') as f:
        json.dump(model, f)

def predict_image(img_path, model_path):
    """Распознавание изображения"""
    with open(model_path, 'r') as f:
        model = json.load(f)

    img = Image.open(img_path).convert('L')
    img_array = (np.array(img) > 128).astype(int)
    signature = normalize_signature(trace_image(img_array))

    for class_name, samples in model.items():
        if signature in samples:
            return class_name
    return "Unknown"


def calculate_accuracy(test_data_dir, model_path):
    """
    Вычисляет точность распознавания на тестовом наборе данных
    :param test_data_dir: Путь к тестовой директории с классами
    :param model_path: Путь к сохраненной модели
    :return: Точность в процентах, количество верных/общее
    """
    correct = 0
    total = 0

    # Получаем список классов
    classes = [d for d in os.listdir(test_data_dir)
              if os.path.isdir(os.path.join(test_data_dir, d))]

    for true_class in classes:
        class_dir = os.path.join(test_data_dir, true_class)
        if not os.path.isdir(class_dir):
            continue

        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)

            # Предсказываем класс
            predicted = predict_image(img_path, model_path)

            # Сравниваем с истинным
            if predicted == true_class:
                correct += 1
            total += 1

    accuracy = (correct / total) * 100 if total > 0 else 0
    return accuracy, f"{correct}/{total}"

In [29]:
# Обучение
train_model("train_comp", "model.json")

Обучение модели: 100%|██████████| 2360/2360 [00:02<00:00, 1134.94img/s]


In [30]:
# Распознавание
result = predict_image("test_comp/1/0002.png", "model.json")
print("Результат:", result)

Результат: Unknown


In [33]:
accuracy, ratio = calculate_accuracy("test_comp", "model.json")
print(f"Точность: {accuracy:.2f}% ({ratio})")

Точность: 5.80% (66/1137)
